In [1]:
import torch

In [2]:
# Root directory for dataset
dataroot = "vehicle data 4 model/train/car"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 200

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [3]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
from torchvision.models import resnet34

resnet_model_carModel = resnet34(pretrained = True).to(device)
resnet_model_vtype = resnet34(pretrained = True).to(device)
gan_generator = Generator(ngpu = 1).to(device)


resnet_model_vtype.fc = nn.Sequential (
    nn.Linear(512,512),
    nn.Dropout(0.2),
     nn.Linear(512,256),
    nn.Linear(256,20), # -> len(labels) labels according to labels
)

resnet_model_carModel.fc = nn.Sequential (
    nn.Linear(512,512),
    nn.Dropout(0.2),
     nn.Linear(512,256),
    nn.Linear(256,7), # -> len(labels) labels according to labels
)


resnet_model_vtype.fc

resnet_model_carModel.fc

c:\Users\manki\OneDrive\Documents\knowledge distillation 2_new model\env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\manki\OneDrive\Documents\knowledge distillation 2_new model\env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Linear(in_features=512, out_features=512, bias=True)
  (1): Dropout(p=0.2, inplace=False)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): Linear(in_features=256, out_features=7, bias=True)
)

In [6]:
# Assuming ResNetModel and GANGenerator are your model classes

resnet_model_carModel.load_state_dict(torch.load('Car_model_model.pth'))

resnet_model_vtype.load_state_dict(torch.load('vehicle_type_model.pth'))
gan_generator.load_state_dict(torch.load('generator.pth'))


resnet_model_carModel.eval()
resnet_model_vtype.eval()

gan_generator.eval()


C:\Users\manki\AppData\Local\Temp\ipykernel_25860\4126249325.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  resnet_model_carModel.load_state_dict(torch.load('Car_model_

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import numpy as np

transform = transforms.Compose([ transforms.Resize((112, 112)),
                                 transforms.ToTensor()
])


correct = 0

fixed_noise = torch.randn(200, nz, 1, 1, device=device)

img_list = []

fake = gan_generator(fixed_noise).detach().cpu()

resize_transform = transforms.Resize((112, 112))
resized_fake = torch.stack([resize_transform(img) for img in fake])

img_list.append(vutils.make_grid(resized_fake, padding=2, normalize=True))

''' 
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

'''



' \nplt.figure(figsize=(8,8))\nplt.axis("off")\nplt.title("Fake Images")\nplt.imshow(np.transpose(img_list[-1],(1,2,0)))\nplt.show()\n\n'

In [8]:
resnet_model_vtype.to(device)

output = resnet_model_vtype(resized_fake.view(-1,3,112,112).to(device))


softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(output)

# Get the predicted class for each image
_, predicted_classes = torch.max(probabilities, 1)

#print(predicted_classes)

count = torch.sum(predicted_classes == 5).item()
print(count)


152


In [9]:
resnet_model_carModel.to(device)

output = resnet_model_carModel(resized_fake.view(-1,3,112,112).to(device))


softmax = torch.nn.Softmax(dim=1)
probabilities = softmax(output)

# Get the predicted class for each image
_, predicted_classes = torch.max(probabilities, 1)

#print(predicted_classes)

count = torch.sum(predicted_classes == 3).item()
print(count)

183
